In [1]:
#Load packages

import numpy as np
import pandas as pd
import json


In [2]:
#Import volume datasets

e_roads = json.load(open(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\All_Roads\JSON\volume_E.json"))
r_roads = json.load(open(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\All_Roads\JSON\volume_R.json"))
f_roads = json.load(open(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\All_Roads\JSON\volume_F.json"))



In [3]:
#Function that flattens JSON formatted data from Vegvesen API

def flatten(item):
    return [
        {
            "id": item["data"]["trafficData"]["trafficRegistrationPoint"]["id"],
            "name": item["data"]["trafficData"]["trafficRegistrationPoint"]["name"],
            "lat": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lat"],
            "lon": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lon"],
            "volume": month["total"]["volume"]["average"] if month and month["total"] and month["total"]["volume"] else np.nan,
            "year":month["year"] if month else np.nan,
            "month": month["month"] if month else np.nan,
            "heading1": byDirection["heading"] if byDirection["heading"] else np.nan,
            "volume_heading1": byDirection["total"]["volume"]["average"] if byDirection["total"]["volume"] else np.nan
        }
        for month in item["data"]["trafficData"]["volume"]["average"]["daily"]["byMonth"] #IF Remove byMonth -> New error, string must be integers 
        if month

        for byDirection in month["byDirection"]
        if byDirection
    ]

In [4]:
#Execute flatten function on data

data_e = []
for item in e_roads:
    data_e.extend(flatten(item))

data_r = []
for item in r_roads:
    data_r.extend(flatten(item))

    data_f = []
for item in f_roads:
    data_f.extend(flatten(item))


In [5]:
#Convert to dataframe

df_e = pd.DataFrame(data_e)
df_f = pd.DataFrame(data_f)
df_r = pd.DataFrame(data_r)

In [6]:
#Display dataframe 

df_r.head()
df_f.head()
df_e.head()


,id,name,lat,lon,volume,year,month,heading1,volume_heading1
0,65823V1668921,FJØSDALEN,67.98167,13.175464,469.0,2018,1,Leknes,235
1,65823V1668921,FJØSDALEN,67.98167,13.175464,469.0,2018,1,Reine,234
2,65823V1668921,FJØSDALEN,67.98167,13.175464,706.0,2018,2,Leknes,353
3,65823V1668921,FJØSDALEN,67.98167,13.175464,706.0,2018,2,Reine,352
4,65823V1668921,FJØSDALEN,67.98167,13.175464,760.0,2018,3,Leknes,381


In [7]:
#Check amount of registration points in the dataset

unique_id_e=df_e["id"].nunique()
print("E registration points:", unique_id_e)

unique_id_r=df_r["id"].nunique()
print("R registration points:", unique_id_r)

unique_id_f=df_f["id"].nunique()
print("F registration points:", unique_id_f)

total_unique= unique_id_f + unique_id_r + unique_id_e

print("There is a total of" , total_unique , "traffic registration points in the dataset")

E registration points: 623
R registration points: 382
F registration points: 1239
There is a total of 2244 traffic registration points in the dataset


In [8]:
# Concatenate the dataframes vertically

all_points = pd.concat([df_e, df_r, df_f])

# Reset the index of the concatenated dataframe
all_points = all_points.reset_index(drop=True)


In [9]:
all_points.head()

,id,name,lat,lon,volume,year,month,heading1,volume_heading1
0,65823V1668921,FJØSDALEN,67.98167,13.175464,469.0,2018,1,Leknes,235
1,65823V1668921,FJØSDALEN,67.98167,13.175464,469.0,2018,1,Reine,234
2,65823V1668921,FJØSDALEN,67.98167,13.175464,706.0,2018,2,Leknes,353
3,65823V1668921,FJØSDALEN,67.98167,13.175464,706.0,2018,2,Reine,352
4,65823V1668921,FJØSDALEN,67.98167,13.175464,760.0,2018,3,Leknes,381


In [10]:
#Check amount of registration points in the concatenated dataframe

unique_ids=all_points["id"].nunique()
print("All registration points:", unique_ids)

All registration points: 2244


In [11]:
# Print the data types of the columns

print(all_points.dtypes)

id                  object
name                object
lat                float64
lon                float64
volume             float64
year                 int64
month                int64
heading1            object
volume_heading1      int64
dtype: object


In [12]:
# Replace NaN with 0 in the 'volume' column
all_points['volume'].fillna(0, inplace=True)
all_points['volume']=all_points['volume'].round().astype('int64')

In [13]:
# Count the occurrences where 'volume' is equal to 0 or missing
count = all_points['volume'].isnull().sum() + (all_points['volume'] == 0).sum()

# print the count
print(count)

295


In [14]:
# use the value_counts() method to count the occurrences of each month
month_counts = all_points['month'].value_counts()

# print the results
for month, count in month_counts.iteritems():
    print(f'{month}: {count}')

12: 4333
11: 4324
10: 4298
6: 4273
9: 4270
5: 4269
7: 4267
8: 4267
1: 4259
4: 4252
2: 4243
3: 4228


C:\Users\anton\AppData\Local\Temp/ipykernel_2232/12571770.py:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for month, count in month_counts.iteritems():


In [15]:
# define the file path where you want to save the file
file_path = r'C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\All_Roads\CSV\all_road_volume_org.csv'

# save the DataFrame as a CSV file in the specified folder
all_points.to_csv(file_path, index=False)

In [39]:
df = df.drop_duplicates(subset=["id", "month"])


In [41]:
df.head()


df = df.drop(columns=["heading1", "volume_heading1"])


In [42]:
df.head()

df["volume1"]=df["volume"]/2
df["volume2"]=df["volume"]/2



In [ ]:
# check if column 'A' contains numeric values
if pd.api.types.is_numeric_dtype(df['volume1']):
    print('Column A contains numeric values')
else:
    print('Column A does not contain numeric values')

# view the rows in column 'A' with non-numeric values
non_numeric_rows = df.loc[~pd.to_numeric(df['volume1'], errors='coerce').notnull()]
print(non_numeric_rows)

,id,name,lat,lon,volume,year,month,volume1,volume2
0,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13023,2018,1,6511.5,6511.5
2,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13998,2018,2,6999.0,6999.0
4,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13878,2018,3,6939.0,6939.0
6,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14288,2018,4,7144.0,7144.0
8,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14669,2018,5,7334.5,7334.5


In [46]:
df['volume1']=df['volume1'].round().astype(int)
df['volume2']=df['volume2'].round().astype(int)

In [47]:
df.head()

,id,name,lat,lon,volume,year,month,volume1,volume2
0,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13023,2018,1,6512,6512
2,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13998,2018,2,6999,6999
4,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13878,2018,3,6939,6939
6,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14288,2018,4,7144,7144
8,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14669,2018,5,7334,7334


In [49]:
df.to_excel("weekly_data_xl.xlsx")